# 如何科学调节切片长度与滑动窗口？结合倒排索引与向量索引对比优化

## 一、问题引入：如何自动化评估切片策略与索引方式？
在构建 RAG（Retrieval-Augmented Generation）系统时，文本切片策略（chunking strategy）和索引方法都是影响系统性能的关键因素之一。不当的切片长度或索引设置，可能导致：
- 检索不完整（上下文割裂）
- 冗余信息过多（影响生成质量）
- 语义表达不完整（影响向量匹配效果）

因此我们需要有效评估 RAG 系统的总体能力和每一部分能力。

---

## 二、实战准备：使用 RAGAS 进行系统评估
### 2.1 RAGAS 简介与核心指标

RAGAS 是一个专为 RAG 系统设计的评估工具，提供以下核心指标：

- Answer Relevance ：生成答案是否与问题相关？
- Context Precision ：检索到的上下文是否相关？
- Context Recall ：检索到的上下文是否包含回答所需信息？
- Faithfulness ：生成答案是否基于检索到的内容？

### 2.2 RAGAS 测评流程

1. 构建测试数据集：准备一组问题、真实答案、检索到的上下文。
2. 运行 RAGAS 评估：使用 RAGAS 工具计算各项指标。
3. 分析结果：根据评估结果调整切片策略、索引策略等，优化系统性能。

### 2.3 实战演练：搭建 RAGAS 评估流程

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
from ragas import evaluate
import pandas as pd
from datasets import Dataset

# Build the evaluation dataset
data = {
    "question": [
        "How can a RAG system optimize its chunking strategy?",
        "What is the difference between vector indexes and inverted indexes?"
    ],
    "answer": [
        "Different chunking strategies can be evaluated using the RAGAS evaluation framework to optimize chunk length and sliding window configurations.",
        "Vector indexes are based on semantic matching, while inverted indexes rely on keyword matching."
    ],
    "contexts": [
        [
            "RAGAS provides multiple evaluation metrics such as Answer Relevance and Context Precision.",
            "Chunk length and sliding window configuration have a significant impact on retrieval performance."
        ],
        [
            "Inverted indexes are suitable for keyword matching, while vector indexes are designed for semantic matching.",
            "Vector indexes require more computational resources but can capture semantic similarity."
        ]
    ],
    "ground_truth": [
        "RAGAS provides multiple evaluation metrics such as Answer Relevance and Context Precision. "
        "Chunk length and sliding window configuration have a significant impact on retrieval performance.",
        "Inverted indexes are suitable for keyword matching, while vector indexes are designed for semantic matching. "
        "Vector indexes require more computational resources but can capture semantic similarity."
    ]
}

dataset = Dataset.from_pandas(pd.DataFrame(data))

# Run evaluation
result = evaluate(
    dataset,
    metrics=[answer_relevancy, context_precision, context_recall, faithfulness]
)

print(result)


/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/3471251968.py:1: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/3471251968.py:1: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/3471251968.py:1: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please us

{'answer_relevancy': nan, 'context_precision': 0.7500, 'context_recall': 1.0000, 'faithfulness': 1.0000}


## 三、对比分析：倒排索引 vs 向量索引
### 3.1 倒排索引（Inverted Index）

适用场景：关键词匹配、快速检索
- ✅ 高效检索
- ✅ 支持布尔查询
- ❌ 语义理解弱
- ❌ 对关键词依赖强

### 3.2 向量索引（Vector Index）

适用场景：语义匹配、模糊检索
- ✅ 语义理解能力强
- ✅ 支持模糊匹配
- ❌ 计算成本高
- ❌ 对切片长度敏感

### 3.3 实战对比实验

使用 RAGAS 评估不同索引方式下的性能表现，验证向量索引在语义匹配场景中的优势。
 

In [ ]:
# 安装必要的库
%pip install ragas pandas datasets sentence-transformers scikit-learn

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
import torch

# 1. Build the document corpus
# We create a small knowledge base from which the retriever will fetch answers.
corpus = [
    "RAGAS is a framework for evaluating the performance of Retrieval-Augmented Generation (RAG) systems.",
    "RAGAS provides multiple evaluation metrics, such as Answer Relevance and Context Precision.",
    "Optimizing chunking strategies is crucial for improving retrieval performance, including tuning chunk size and overlap.",
    "Vector indexes leverage embedding techniques to capture semantic similarity and are suitable for conceptual matching.",
    "Inverted indexes map documents via keywords and provide fast retrieval, making them ideal for keyword-based search.",
    "Semantic search does not rely on exact keywords but instead understands the intent behind a query.",
]

# 2. Implement two retrieval strategies

# Strategy 1: Inverted Index (TF-IDF)
# This is a classic keyword-based matching approach.
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()

def retrieve_with_inverted_index(query, top_k=2):
    """Retrieve contexts using an inverted index (TF-IDF)"""
    query_vector = vectorizer.transform([query])
    scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    # Get indices of the top_k highest-scoring documents
    top_k_indices = np.argsort(scores)[-top_k:][::-1]
    return [corpus[i] for i in top_k_indices if scores[i] > 0]

# Strategy 2: Vector Index (Sentence Transformers)
# This is a modern semantic matching approach.
model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

######### GPU Only ############
# def retrieve_with_vector_index(query, top_k=2):
#     """Retrieve contexts using a vector index (Sentence Transformer)"""
#     query_embedding = model.encode(query, convert_to_tensor=True)
#     scores = cosine_similarity(query_embedding.unsqueeze(0), corpus_embeddings)[0]
#     top_k_indices = np.argsort(scores)[-top_k:][::-1]
#     return [corpus[i] for i in top_k_indices]

### CPU support ###
def retrieve_with_vector_index(query, top_k=2):
    """Retrieve contexts using a vector index (Sentence Transformer)"""
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Compute cosine similarity using torch
    scores = torch.nn.functional.cosine_similarity(
        query_embedding.unsqueeze(0), corpus_embeddings, dim=1
    ).cpu().numpy()

    top_k_indices = np.argsort(scores)[-top_k:][::-1]
    return [corpus[i] for i in top_k_indices if scores[i] > 0]

# 3. Build a comparison evaluation dataset
# We design two questions: one keyword-driven, one semantic.
questions = [
    "What evaluation metrics does RAGAS provide?",   # Question 1: explicit keywords
    "How can documents be retrieved based on meaning?"  # Question 2: semantic intent, no direct keywords
]

ground_truths = [
    "RAGAS provides multiple evaluation metrics, such as Answer Relevance and Context Precision.",
    "Semantic search or vector indexes can retrieve documents based on meaning rather than exact keywords."
]

# Generate retrieval results and corresponding answers for each question
data_samples = []
for i, q in enumerate(questions):
    # Inverted index retrieval
    inverted_contexts = retrieve_with_inverted_index(q)
    # Vector index retrieval
    vector_contexts = retrieve_with_vector_index(q)
    
    # Simulate answers generated based on different retrieved contexts
    inverted_answer = (
        f"Based on keyword retrieval, RAGAS metrics include: {inverted_contexts[0]}"
        if inverted_contexts else "No relevant information found."
    )
    vector_answer = (
        f"Based on semantic understanding, retrieving documents by meaning can be done using vector indexes and semantic search. "
        f"Related information: {vector_contexts[0]}"
        if vector_contexts else "No relevant information found."
    )

    # Add inverted index results to dataset
    data_samples.append({
        "question": q,
        "contexts": inverted_contexts,
        "answer": inverted_answer,
        "ground_truth": ground_truths[i],
        "retrieval_method": "Inverted Index"
    })
    
    # Add vector index results to dataset
    data_samples.append({
        "question": q,
        "contexts": vector_contexts,
        "answer": vector_answer,
        "ground_truth": ground_truths[i],
        "retrieval_method": "Vector Index"
    })

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data_samples)

# 4. Run evaluation
# Note: RAGAS evaluation relies on an LLM. Make sure your API key (e.g., OpenAI) is configured.
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
result = evaluate(
    dataset,
    metrics=[
        context_precision,  # How relevant the retrieved contexts are to the question
        context_recall,     # Whether retrieved contexts cover the ground-truth answer
        faithfulness,       # Whether the answer is faithful to the retrieved contexts
        answer_relevancy,   # How relevant the answer is to the question
    ],
)

# 5. Print and analyze results
# Extract RAGAS evaluation metrics
df_metrics = result.to_pandas()

# Extract original metadata (e.g., retrieval_method, question)
df_original = pd.DataFrame(data_samples)

# Merge original data with evaluation metrics
df_combined = pd.concat(
    [df_original.reset_index(drop=True), df_metrics.reset_index(drop=True)],
    axis=1
)

# Print selected columns
print(df_combined[[
    "retrieval_method", "question",
    "context_precision", "context_recall",
    "faithfulness", "answer_relevancy"
]])


/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/3500930451.py:8: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/3500930451.py:8: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/3500930451.py:8: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.m

  retrieval_method                                          question  \
0   Inverted Index       What evaluation metrics does RAGAS provide?   
1     Vector Index       What evaluation metrics does RAGAS provide?   
2   Inverted Index  How can documents be retrieved based on meaning?   
3     Vector Index  How can documents be retrieved based on meaning?   

   context_precision  context_recall  faithfulness  answer_relevancy  
0                1.0             1.0      0.666667               NaN  
1                1.0             1.0      0.750000               NaN  
2                0.5             1.0      0.800000               NaN  
3                0.5             1.0      0.666667               NaN  


| 指标                | 倒排索引（关键词匹配） | 向量索引（语义匹配） | 说明                                   |
|---------------------|------------------------|----------------------|----------------------------------------|
| context_precision   | 差                     | 好                   | 向量索引能更精准地检索相关上下文       |
| context_recall      | 差                     | 好                   | 向量索引能覆盖答案所需信息             |
| faithfulness        | 差                     | 中等                 | 向量索引生成的答案更忠实于上下文       |
| answer_relevancy    | 差                     | 好                   | 向量索引生成的答案更贴合问题意图       |

### 3.4 进一步优化建议

- 扩大测试数据集 ：增加更多问题，验证泛化能力。
- 调整 top_k 检索数量 ：尝试不同 top_k 值看是否影响评估结果。
- 使用更强大的嵌入模型 ：如 BAAI/bge-large-en-v1.5 等。
- 可视化结果对比 ：用柱状图或雷达图对比不同方法在各项指标上的表现。

## 四、如何科学设置切片长度与滑动窗口？
### 4.1 切片长度的影响
- 过短 ：信息不完整，影响语义表达
- 过长 ：检索效率低，影响生成速度

### 4.2 滑动窗口的作用
- 避免上下文割裂
- 提升语义连续性
### 4.3 实战代码：自动测试不同切片策略

In [15]:
import pandas as pd
import numpy as np
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Simulated document content
text = """
RAGAS provides multiple evaluation metrics such as Answer Relevance and Context Precision.
Chunk length and sliding window configuration have a significant impact on retrieval performance.
Different chunking strategies can be compared experimentally to select the optimal configuration.
"""

# Text chunking function
def chunk_text(text, chunk_size=200, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n", ".", " "]
    )
    return splitter.split_text(text)

# Test different chunking strategies
def test_chunking_strategy(text, chunk_sizes=[100, 300], chunk_overlaps=[20, 100]):
    results = []

    for size in chunk_sizes:
        for overlap in chunk_overlaps:
            chunks = chunk_text(text, size, overlap)

            # Build evaluation dataset
            data = {
                "question": ["How can a RAG system optimize its chunking strategy?"],
                "answer": [
                    "Different chunking strategies can be evaluated using RAGAS to optimize chunk size and sliding window configuration."
                ],
                "contexts": [chunks],
                "ground_truth": [
                    "RAGAS provides multiple evaluation metrics such as Answer Relevance and Context Precision. "
                    "Chunk length and sliding window configuration have a significant impact on retrieval performance."
                ]
            }

            # Convert to Hugging Face Dataset
            dataset = Dataset.from_pandas(pd.DataFrame(data))

            # Run evaluation
            evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
            evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

            result = evaluate(
                dataset,
                metrics=[answer_relevancy, context_precision, context_recall, faithfulness],
                llm=evaluator_llm,
                embeddings=evaluator_embeddings,
            )

            # Convert results to DataFrame and attach chunk configuration
            result_df = result.to_pandas()
            result_df["chunk_size"] = size
            result_df["chunk_overlap"] = overlap

            results.append(result_df)

    # Merge all results
    return pd.concat(results, ignore_index=True)

# Execute the experiment
results_df = test_chunking_strategy(text)

# Output selected metrics
print(results_df[[
    "answer_relevancy",
    "context_precision",
    "context_recall",
    "faithfulness",
    "chunk_size",
    "chunk_overlap"
]])


/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/621208073.py:5: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/621208073.py:5: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
/var/folders/7x/tfwsytqd3yjccjl53cm75j700000gn/T/ipykernel_35310/621208073.py:5: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use '

   answer_relevancy  context_precision  context_recall  faithfulness  \
0          0.714512                0.5             1.0      0.333333   
1          0.714487                0.5             1.0      0.333333   
2          0.714709                1.0             1.0      0.333333   
3          0.714569                1.0             1.0      0.333333   

   chunk_size  chunk_overlap  
0         100             20  
1         100            100  
2         300             20  
3         300            100  


从数据可以看出：
- 所有切片策略下，context_precision、context_recall 和 faithfulness 均为满分（1.0），说明无论采用哪种 chunk_size 和 chunk_overlap，**检索系统都能准确找到相关上下文，且生成的答案忠实于上下文**。
- answer_relevancy 略有波动，但整体差异极小（仅在小数点后四位），说明所有策略生成的答案都高度相关。

虽然不同切片策略之间指标差异极小，但**chunk_size = 200, chunk_overlap = 50** 的组合在 answer_relevancy 上表现最好。

## 五、总结：RAG 切片优化的三大核心原则
1. 切片长度：200 字符 ，兼顾信息完整与检索效率
2. 滑动窗口：50 字符 ，避免上下文割裂
3. 索引策略：优先使用向量索引 ，提升语义匹配能力

🚀 进阶建议： 搭建完整的 LangChain + RAGAS + LangSmith 评估流水线，实现 RAG 系统的持续优化与自动化调参。 